In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
def load_all_models():
    all_models = []
    model_names = ['model1.h5', 'model2.h5', 'model3.h5']
    for model_name in model_names:
        filename = os.path.join('gdrive/MyDrive/Datasets', model_name)
        model = tf.keras.models.load_model(filename)
        all_models.append(model)
        print('loaded:', filename)
    return all_models

In [ ]:
def ensemble_model(models):
    for i, model in enumerate(models):
        for layer in model.layers:
            layer.trainable = False
    ensemble_visible = [model.input for model in models]
    ensemble_outputs = [model.output for model in models]
    merge1 = tf.keras.layers.concatenate(ensemble_outputs)
    merge2 = tf.keras.layers.Dense(18, activation='relu')(merge1)
    output = tf.keras.layers.Dense(9, activation='softmax')(merge2)
    model = tf.keras.models.Model(inputs=ensemble_visible, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=["accuracy"])
    return model

In [ ]:
models = load_all_models()
model = ensemble_model(models)

loaded: gdrive/MyDrive/Datasets/model1.h5
loaded: gdrive/MyDrive/Datasets/model2.h5
loaded: gdrive/MyDrive/Datasets/model3.h5


In [ ]:
import cv2

In [ ]:
from pandas._libs.tslibs.tzconversion import tz_localize_to_utc
def generator(samples, batch_size=32,resize=224):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[image1_filename,label1], [image2_filename,label2],...].
    """
    
    num_samples = len(samples)
    while True: # Loop forever so the generator never terminates

        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size <= num_samples]
        
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = samples[offset:offset+batch_size].values.tolist()
            
            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            
            # For each example
            for batch_sample in batch_samples:
                # Load image (X) and label (y)
                img_name = batch_sample[0]
                label = batch_sample[1:]
                img =  cv2.imread(os.path.join(img_name))
                
                # apply any kind of preprocessing
                img = cv2.resize(img,(resize,resize))
                # Add example to arrays
                X_train.append(img)
                y_train.append(label)
                
            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            y_train = np.array(y_train)

            # The generator-y part: yield the next training batch            
            yield [X_train,X_train,X_train], y_train
            X_train = []
            y_train = []

In [ ]:
sample = pd.get_dummies(train['Label'])
train = train.drop('Label', axis = 1)
train = train.join(sample)

In [ ]:
train.head()

,img_IDS,1,2,3,4,5,6,7,8,9
0,/content/Images/Images/ImageID_33HMDMJ3.jpg,1,0,0,0,0,0,0,0,0
1,/content/Images/Images/ImageID_V5POEX4O.jpg,0,1,0,0,0,0,0,0,0
2,/content/Images/Images/ImageID_89CCCCW6.jpg,0,0,1,0,0,0,0,0,0
3,/content/Images/Images/ImageID_GNJXOWX5.jpg,0,0,0,1,0,0,0,0,0
4,/content/Images/Images/ImageID_7Q9LOP7R.jpg,0,0,0,0,1,0,0,0,0


In [ ]:
train.head()

,img_IDS,Label
0,/content/Images/Images/ImageID_33HMDMJ3.jpg,1
1,/content/Images/Images/ImageID_V5POEX4O.jpg,2
2,/content/Images/Images/ImageID_89CCCCW6.jpg,3
3,/content/Images/Images/ImageID_GNJXOWX5.jpg,4
4,/content/Images/Images/ImageID_7Q9LOP7R.jpg,5


In [ ]:
train_datagen = generator(train[:4992],batch_size=32)
val_datagen = generator(train[5001:],batch_size=32)

In [ ]:
for i in range(157):
  x,y = next(train_datagen)
  print (y.shape)

In [ ]:
for _ in range(len(model.input)):
  print(train_ds)

<BatchDataset shapes: ((None, 224, 224, 3), (None, 9)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 224, 224, 3), (None, 9)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 224, 224, 3), (None, 9)), types: (tf.float32, tf.float32)>


In [ ]:
X_train = [train_ds for _ in range(len(model.input))]
X_test = [val_ds for _ in range(len(model.input))]

In [ ]:
epochs = 3
history = model.fit(train_datagen,
                    batch_size=32,
                    epochs=epochs, 
                    validation_data=val_datagen)

Epoch 1/3
   1245/Unknown - 774s 603ms/step - loss: 0.3977 - accuracy: 0.9337

In [ ]:
path = '/content/Images/Images/ImageID_00A5MDP9.jpg'
img_height=224
img = tf.keras.utils.load_img(
    path, target_size=(img_height, img_height)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

prediction = model.predict((img_array,img_array,img_array))
score = tf.nn.softmax(prediction[0])


print(
    "Model1 : This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(np.argmax(score), 100 * np.max(score))
)

Model1 : This image most likely belongs to 4 with a 13.18 percent confidence.
